<a href="https://colab.research.google.com/github/glorivaas/Risk_Measures/blob/main/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Numerical integration of SDEs

### Exercise 1
Implement numerical integration of the cev model.

In [3]:

import numpy as np
import numpy.typing as npt


def cev_model_mc(
    s0: float,
    t: float,
    vol: float,
    gamma: float,
    mu: float,
    n_paths: int,
    n_steps: int,
    use_millstein_scheme: bool = False
) -> npt.NDArray[np.float64]:
    """Returns array of shape (n_paths, n_steps) containing paths of the Cev model."""
    dt = t / n_steps
    paths = np.zeros((n_paths, n_steps + 1))
    paths[:, 0] = s0

    for i in range(n_steps):
        s = paths[:, i]
        dW = np.random.normal(0, np.sqrt(dt), size=n_paths)
        diffusion = vol * np.power(np.maximum(s, 0), gamma)

        if use_millstein_scheme:
            derivative = vol * gamma * np.power(np.maximum(s, 0), gamma - 1)
            paths[:, i + 1] = s + mu * s * dt + diffusion * dW + 0.5 * diffusion * derivative * (dW**2 - dt)
        else:
            paths[:, i + 1] = s + mu * s * dt + diffusion * dW

    return paths

### Exercise 2

For the MC scheme results, calculate mean and the confidence interval at specified level, using the unbiased variance estimator and assuming that we can use CLT approximation.

In [4]:
def mc_mean_confidence_interval(mc_results: list[float], confidence_level=0.95) -> tuple[float, tuple[float, float]]:
    """Returns mean and confidence interval at specified level."""
    data = np.array(mc_results)
    n = len(data)
    mean = np.mean(data)
    std_err = np.std(data, ddof=1) / np.sqrt(n)
    z = stats.norm.ppf(0.5 + confidence_level / 2)

    margin = z * std_err
    conf_interval = (mean - margin, mean + margin)

    return mean, conf_interval

### Exercise 3

Use hte results of exercise 1 and 2 to calculate the price and delta of the European call option in the CEV model.

Calculate the price of the following contract.
- s0 = 100.0
- t = 3.0
- vol = 0.3
- gamma = 1.5
- mu = 0.2
- strike in [70, 80, 90, 100]
- moneyness (K/s0) in [0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

Tweak the simulation parameters s.t. the price conf. interval at level 99% has width of at most 1 basis point for every strike.

Find the implied volatilities of these options and calculate the delta in the BS model using the results of Lab5.

In [6]:
def european_call_cev_price(
    s0: float,
    t: float,
    vol: float,
    gamma: float,
    r: float,
    strike: float,
    n_paths: int,
    n_steps: int,
    confidence_level: float = 0.99,
    use_millstein_scheme: bool = False
) -> tuple[float, tuple[float, float]]:
    """Returns price and confidence interval at specified level."""
    paths = cev_model_mc(
        s0=s0,
        t=t,
        vol=vol,
        gamma=gamma,
        mu=r,
        n_paths=n_paths,
        n_steps=n_steps,
        use_millstein_scheme=use_millstein_scheme
    )

    payoffs = np.maximum(paths[:, -1] - strike, 0)
    discounted_payoffs = np.exp(-r * t) * payoffs
    return mc_mean_confidence_interval(discounted_payoffs.tolist(), confidence_level)

def bs_call_price(s0, k, t, r, sigma):
    """Black-Scholes price of a European call option."""
    if t <= 0 or sigma <= 0:
        return max(s0 - k, 0.0)
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma**2) * t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)
    return s0 * norm.cdf(d1) - k * np.exp(-r * t) * norm.cdf(d2)

def implied_volatility(
    price: float,
    s0: float,
    t: float,
    r: float,
    strike: float,
) -> float:
    """Returns Black Scholes implied volatility for given price, strike, and time to maturity."""
    def objective(sigma):
        return bs_call_price(s0, strike, t, r, sigma) - price

    return brentq(objective, 1e-6, 3.0)  # Search in a reasonable volatility range

